In [21]:
import json
from openai import OpenAI
import time
import os
import requests


client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY")
)

In [22]:
assistant_id="asst_KTFlL51Og2srFF5bqmaaKM9k"


In [23]:

def go_to_location(location):

    time.sleep(1)
    return f"""i have reached the {location}.\n"""

go_to_location("kitchen")


'i have reached the kitchen.\n'

In [24]:
assistant=client.beta.assistants.retrieve(
    assistant_id=assistant_id
)

assistant


Assistant(id='asst_KTFlL51Og2srFF5bqmaaKM9k', created_at=1701638723, description=None, file_ids=[], instructions='you are a   service robot in the real world. you can have small talk with the user you can go to different rooms in the home based on the user input.', metadata={}, model='gpt-3.5-turbo', name='robot assistant', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='go_to_location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'name of the room like kitchen,bedroom,bathroom'}}, 'required': ['location']}, description='go to location as requested by user'), type='function')])

In [25]:
thread=client.beta.threads.create()
thread

Thread(id='thread_csaKPMBeHt3W0y7ZcXsj3WfO', created_at=1701713822, metadata={}, object='thread')

In [26]:
run=client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="go to the kitchen"
)

run

Run(id='run_zy6rIa8YpRqji1a33ktqpO99', assistant_id='asst_KTFlL51Og2srFF5bqmaaKM9k', cancelled_at=None, completed_at=None, created_at=1701713825, expires_at=1701714425, failed_at=None, file_ids=[], instructions='go to the kitchen', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_csaKPMBeHt3W0y7ZcXsj3WfO', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='go_to_location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'name of the room like kitchen,bedroom,bathroom'}}, 'required': ['location']}, description='go to location as requested by user'), type='function')])

In [27]:
run=client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

run

Run(id='run_zy6rIa8YpRqji1a33ktqpO99', assistant_id='asst_KTFlL51Og2srFF5bqmaaKM9k', cancelled_at=None, completed_at=None, created_at=1701713825, expires_at=1701714425, failed_at=None, file_ids=[], instructions='go to the kitchen', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_aE8yEw51axIJnyho9oZ0g22Y', function=Function(arguments='{\n  "location": "kitchen"\n}', name='go_to_location'), type='function')]), type='submit_tool_outputs'), started_at=1701713825, status='requires_action', thread_id='thread_csaKPMBeHt3W0y7ZcXsj3WfO', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='go_to_location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'name of the room like kitchen,bedroom,bathroom'}}, 'required': ['location']}, description='go to location as requested by user'

In [28]:
def get_outputs_for_tool_call(tool_call):
    location=json.loads(tool_call.function.arguments)["location"]
    location_details=go_to_location(location=location)
    return{
        "tool_call_id":tool_call.id,
        "output":location_details
    }

tools_calls=run.required_action.submit_tool_outputs.tool_calls
tool_outputs=map(get_outputs_for_tool_call,tools_calls)
tool_outputs=list(tool_outputs)
tool_outputs


[{'tool_call_id': 'call_aE8yEw51axIJnyho9oZ0g22Y',
  'output': 'i have reached the kitchen.\n'}]

In [29]:
run=client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs)

run

Run(id='run_zy6rIa8YpRqji1a33ktqpO99', assistant_id='asst_KTFlL51Og2srFF5bqmaaKM9k', cancelled_at=None, completed_at=None, created_at=1701713825, expires_at=1701714425, failed_at=None, file_ids=[], instructions='go to the kitchen', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1701713825, status='queued', thread_id='thread_csaKPMBeHt3W0y7ZcXsj3WfO', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='go_to_location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'name of the room like kitchen,bedroom,bathroom'}}, 'required': ['location']}, description='go to location as requested by user'), type='function')])

In [30]:
run=client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run

Run(id='run_zy6rIa8YpRqji1a33ktqpO99', assistant_id='asst_KTFlL51Og2srFF5bqmaaKM9k', cancelled_at=None, completed_at=1701713876, created_at=1701713825, expires_at=None, failed_at=None, file_ids=[], instructions='go to the kitchen', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1701713874, status='completed', thread_id='thread_csaKPMBeHt3W0y7ZcXsj3WfO', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='go_to_location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'name of the room like kitchen,bedroom,bathroom'}}, 'required': ['location']}, description='go to location as requested by user'), type='function')])